# Object Detection Starter — Colab/Kaggle Quickstart

> Author : Badr TAJINI

**Academic year:** 2025–2026  
**School:** ECE  
**Course:** Machine Learning & Deep Learning 2 

---

Follow this guide to validate the YOLOv8 starter on a free GPU runtime. You will confirm the environment, install dependencies, run a smoke prediction, and optionally fine-tune/evaluate on coco128.

## 0. Project files

- Option A: `git clone <your_repo_url> od-project`
- Option B: Upload the `od-project` folder via the sidebar (ensure it appears as `/content/od-project`).

Run the next cells afterwards; they will error helpfully if the folder is missing.

### Quick checklist before running code
- Switch the runtime to **GPU (T4)** first.
  - Colab: `Runtime → Change runtime type → GPU → Save`.
  - Kaggle: gear icon → enable **Accelerator**, choose `T4 x1`.
- Wait for the runtime to reconnect and show a GPU badge.
- Execute cells from top to bottom. If a cell fails, fix it and rerun that cell before moving on.

### How to run a cell
- Click the ▶️ button to the left of a cell, or press **Shift+Enter** (Colab) / **Ctrl+Enter** (Kaggle).
- When a cell finishes, the number on the left becomes `[1]`, `[2]`, etc.
- Don’t skip steps—the later ones depend on the earlier setup.

### Step 0 — Confirm the GPU is ready
Run the next cell. You should see GPU name and memory. If you see `nvidia-smi unavailable`, the runtime is still on CPU—return to the checklist, enable GPU, then rerun this cell.

In [1]:
!nvidia-smi || echo "nvidia-smi unavailable (CPU runtime)"


Tue Nov 25 19:54:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 577.03                 Driver Version: 577.03         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   62C    P0             27W /   50W |    2528MiB /   8188MiB |     56%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Step 1 — Point the notebook at the project folder
This cell ensures the current working directory is `od-project`.
If it raises a `FileNotFoundError`, confirm where you uploaded/cloned the folder and adjust the path before rerunning.

In [2]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent.resolve()
elif PROJECT_ROOT.name == "content":
    candidate = PROJECT_ROOT / "od-project"
    if candidate.exists():
        PROJECT_ROOT = candidate.resolve()

if not (PROJECT_ROOT / "src").exists():
    raise FileNotFoundError(
        f"Could not locate project root at {PROJECT_ROOT}. Upload or clone od-project before proceeding."
    )

os.chdir(PROJECT_ROOT)
if str(PROJECT_ROOT / "src") not in sys.path:
    sys.path.append(str(PROJECT_ROOT / "src"))
print(f"Project root: {PROJECT_ROOT}")


Project root: C:\Users\lucas\End-to-End-Deep-Learning-Systems\End-to-End-Deep-Learning-Systems\starters\od-project-starter\od-project


### Step 2 — Install the project requirements
Installs Ultralytics + supporting libraries from `requirements.txt`. First run downloads the YOLO packages, which may take a couple of minutes.
If installation fails (e.g., due to connectivity), rerun this cell before proceeding.

In [3]:
# Install project dependencies listed in requirements.txt
!pip install -r requirements.txt


  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached polars-1.35.2-py3-none-any.whl.metadata (10 kB)
  Using cached ultralytics_thop-2.0.18-py3-none-any.whl.metadata (14 kB)
  Using cached polars_runtime_32-1.35.2-cp39-abi3-win_amd64.whl.metadata (1.5 kB)
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 9.9 MB/s  0:00:00
Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   --------------- ------------------------ 5.0/12.6 MB 25.3 MB/s eta 0:00:01
   ---------------------------------------  12.6/12.6 MB 30.2 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 27.8 MB/s  0:00:00
Using cached polars-1.35.2-py3-none-any.whl (783 kB)
Using cached polars_runtime_32-1.35.2-cp39-abi3-win_amd64.whl (41.3 MB)
Using cached ultralytics_thop-2.0.18-py3-none-any.whl (28 kB)

### Step 3 — Run the smoke test
This loads the pretrained YOLO weights specified in the config and runs a single prediction on a sample image. It writes `outputs/smoke_metrics.json` so you know the toolkit is ready.
If you hit a download error, wait briefly and rerun the cell.

In [4]:
from src import smoke_check

smoke_path = smoke_check.run_smoke("configs/yolo_coco128.yaml")
print(smoke_path.read_text())


Ultralytics 8.3.228  Python-3.10.18 torch-2.5.1 


ValueError: Invalid CUDA 'device=auto' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


## 1. Review smoke-test output
- Confirm the previous cell printed a JSON block showing the number of detections and mean confidence.
- `outputs/smoke_metrics.json` should now appear in the file browser.
- Need only a quick environment check? Stop here. Ready to fine-tune? Continue to Section 2.
- If anything failed, read the message, fix the issue, and rerun the smoke cell before moving on.

## 2. Full training run (optional)
Run these cells when you want to fine-tune YOLOv8 on `coco128` (or your own dataset). On the first run, downloading the dataset weights can take a couple of minutes.

**Before running:**
1. Open `configs/yolo_coco128.yaml` if you want to edit epochs, batch size, or dataset paths.
2. Ensure the runtime still shows a GPU connection.
3. Close other heavy tabs so the GPU memory stays available.

In [ ]:
# Train the model using the default coco128 configuration.
# Ultralytics manages the training loop and saves weights to outputs/train/weights/.
!python src/train.py --config configs/yolo_coco128.yaml


In [ ]:
# Evaluate the best checkpoint on the validation split.
# Results (mAP, precision, recall) are saved to outputs/eval.json.
!python src/evaluate.py --config configs/yolo_coco128.yaml --ckpt outputs/train/weights/best.pt


### Step 4 — What should I see now?
- `outputs/train/weights/best.pt`: best fine-tuned weights.
- `outputs/train/results.csv` and `outputs/train/events/`: Ultralytics training logs.
- `outputs/eval.json`: validation metrics (mAP50, mAP, precision, recall).
- Optional: run `python src/predict.py ...` to generate annotated images.
If any files are missing, scroll up for errors in the training/evaluation cells.

## 3. Mirror this workflow for custom datasets
1. Duplicate this notebook for your dataset (e.g., `01_custom_yolo.ipynb`).
2. Update the config file (e.g., `configs/yolo_custom.yaml`) to point at your `data.yaml`.
3. Swap the train/eval commands to use the custom config and checkpoint paths.
4. Use `python src/predict.py --config <cfg> --ckpt <weights> --source <images>` to test the trained model.

Keep the order (GPU check → install → smoke test → train → evaluate) for reliable, reproducible runs.